In [2]:
#import matplotlib
%matplotlib qt4
#%matplotlib nbagg


In [3]:
import hyperspy.api as hs
import matplotlib.pyplot as plt

In [4]:
import numpy as np

In [5]:
def gauss(sigma, eRange):
    dE = eRange[1]-eRange[0]
    gx = np.arange(-3*sigma,3*sigma, dE)
    gaussian = np.exp(-0.5*(gx/sigma)**2)
    gaussian = gaussian/np.sum(gaussian)
    
    gauss =np.zeros((len(gaussian),1,1,1))
    gauss[:,0,0,0] = gaussian
    return gauss

In [6]:
def smooth(hist, eRange, sigma):
    gaussian = gauss(sigma, eRange)
    
    crop_front = len(gaussian)//2
    if len(gaussian)%2 == 1:
        crop_end = crop_front
    else:
        crop_end = crop_front-1
        
    return signal.convolve(hist, gaussian)[crop_front:-crop_end]

In [7]:
# Plot the DF, draw a ROI. Maneuver/resize it and close the window.
def set_ROI(s, shape="circle", interactive=False):
    """ Selects an interactive region of interst to the signal
    :param s: t
    """
    import hyperspy.api as hs
 
    if s.axes_manager.navigation_dimension < 2:
        axes = "sig"
        x_axis = s.axes_manager[s.axes_manager.signal_indices_in_array[1]]
        y_axis = s.axes_manager[s.axes_manager.signal_indices_in_array[0]]
    else:
        axes = "nav"
        x_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[1]]
        y_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[0]]


    if shape == "circle":
        x = x_axis.axis[round(x_axis.size/2)]
        y = y_axis.axis[round(y_axis.size/2)]

        r_outer = x_axis.axis[round(3*x_axis.size/4)]
    
        sroi = hs.roi.CircleROI(x, y, r=r_outer)
        """
        s.plot()
        sroi= sroi.interactive(s) 
        ss = hs.interactive(f=sroi.sum, event=sroi.events.data_changed)
        """
    elif shape == "ring":
        x = x_axis.axis[round(x_axis.size/2)]
        y = y_axis.axis[round(y_axis.size/2)]

        r_outer = x_axis.axis[round(4*x_axis.size/5)]
        r_inner = x_axis.axis[round(3*x_axis.size/4)]
    
        sroi = hs.roi.CircleROI(x, y, r=r_outer, r_inner=r_inner)
        """
        s.plot()
        sroi= sroi.interactive(s) 
        ss = hs.interactive(f=sroi.sum, event=sroi.events.data_changed)
        """
    else:
        if not shape == "rectangle":
            print("Did not recognize shape, using rectangle")
        x1 = x_axis.axis[1]
        x2 = x_axis.axis[round(x_axis.size/10)]
        y1 = y_axis.axis[1]
        y2 = y_axis.axis[round(y_axis.size/10)]

        sroi = hs.roi.RectangularROI(x1, y1, x2, y2)
        
    if interactive:
        s.plot()
        roi_signal = sroi.interactive(s)
        ss = hs.interactive(f=roi_signal.sum, event=roi_signal.events.data_changed) 
    else:
        roi_signal = sroi(s)
        ss = roi_signal.sum()
        
    return sroi, ss

In [8]:
def signal_by_q(s, q_inner, q_outer):
    """Centers a hollow solid sphere in diffraction space and add signal from whidthin the solid
    :param s: signal1D with 3D diffraction space
    :param q_inner: inner radius of hollow solid sphere (defines the hollow sphere within)
    :param q_outer: outer radius of hollow solid sphere (defines the solid range outside inner sphere)
    :returns: the added signal from the hollow solid sphere
    """
    # Center coordinate
    x, y, = 0, 0
    
    # Get axes
    z_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[2]]
    x_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[1]]
    y_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[0]]
    E_axis = s.axes_manager.signal_axes[0]
    
    # Get shape of EEL spectra
    E_shape = s.axes_manager.signal_shape[0]

    # Crate output signal
    sroi_signal = s.inav[0,0,0]
    sroi_signal.data = np.zeros(E_shape)
    
    # Loop through one axe, select the corresponding 2D region of interst in the other two axes
    for h in range(s.axes_manager.navigation_axes[-1].size):
        r_inner = None
        r_outer = None
        if abs(z_axis.axis[h]) <= abs(q_outer):
            r_outer = np.sqrt(q_outer**2-z_axis.axis[h]**2)
        if abs(z_axis.axis[h]) <= abs(q_inner):
            r_inner = np.sqrt(q_inner**2-z_axis.axis[h]**2)
        
        if r_outer:
            s_temp = s.inav[:,:,h]
            if r_inner:
                sroi = hs.roi.CircleROI(x, y, r=r_outer, r_inner=r_inner)
            else:
                sroi = hs.roi.CircleROI(x, y, r=r_outer)
            s_temp_signal = sroi(s_temp)
            sroi_signal += s_temp_signal.sum()
    return sroi_signal
    

In [17]:
p = hs.load("../Results/smoothed/fermi_-1.65.hspy")
p = p.transpose((2,1,0))
p = p.transpose()
p

<Signal1D, title: Fermi 1.65, dimensions: (71, 71, 71|120)>

In [ ]:
#p.axes_manager

In [ ]:
#p.metadata

# Region of interest

In [22]:
p_roi, p_roi_s = set_ROI(p,shape="circle", interactive=True)
p_roi_s.plot()

# 2D-2D signal

In [ ]:
p2d = p.as_signal2D((0,1))
p2d.plot()

# Comparison

In [18]:
pf = hs.load("../Results/smoothed/fermi_0.10.hspy")
pf = pf.transpose((2,1,0))
pf = pf.transpose()
pf

<Signal1D, title: Fermi 3.4, dimensions: (71, 71, 71|120)>

In [ ]:
pf.plot()

In [ ]:
#pf.axes_manager

In [ ]:
#pf.metadata

In [19]:
p_roi, p_roi_s = set_ROI(p,shape="circle")
pf_roi, pf_roi_s = set_ROI(pf,shape="circle")

In [39]:
color_list = ['#1f77b4','#ff7f0e'] #,pf_roi_s
hs.plot.plot_spectra([p_roi_s,pf_roi_s], legend='auto', color=color_list, legend_loc='best').figure.savefig("../Figures/Schleife2009/four_bands_both.png")

In [ ]:
p2d = pf.as_signal2D((0,1))
p2d.plot()

In [ ]:
hs.plot.plot_spectra([pf_roi_s, p_roi_s])

In [116]:
signals = []
fermi= [1.65, 3.485]
bandGap = 3.3

#folder="ZnO/twobands/truemass/"
#folder="ZnO/twobands/fermitest"
#for level in fermi:

for level in fermi:
    folder = "Schleife2009/unreal_valence/51"
    signal = hs.load('../Results/%s/fermi_%.2f.hspy' %(folder,level-bandGap))
    signals.append(signal)

In [117]:
signals

[<Signal1D, title: Fermi 1.65, dimensions: (51, 51, 51|120)>,
 <Signal1D, title: Fermi 3.485, dimensions: (51, 51, 51|120)>]

In [118]:
eBin = np.linspace(3,5,80)
s_sum = []
for s in signals:
    s_sum.append(s.sum().isig[:-1])
#print(np.argwhere(s_sum[0].data))

In [119]:
test = hs.plot.plot_spectra(s_sum, style="overlap", legend_picking=True, color=color_list, legend='auto', legend_loc='upper left')

In [102]:
plt.savefig("../Figures/Schleife2009_unreal_no_fermi.png")

In [115]:
plt.axvline(x=3.6490043197924233, linestyle='--', color='black')
plt.savefig("../Figures/Schleife2009_unreal_width_fermi.png")

In [ ]:
direct = [3.2999999999999998,
 3.373139744816207,
 3.415420572004225,
 3.4677780787057211,
 3.5255874440286945,
 3.5925589792648283,
 3.635754316053164,
 3.7091988974028101,
 3.7512284972408558]

fermi = [3.0,
        3.35,
        3.4,
        3.45,
        3.5,
        3.55,
        3.6,
        3.65,
        3.7]

for i in range(1,len(direct)):
    print(fermi[i],'\t',direct[i]-fermi[i],'\t',direct[i])

for i in range(len(s_sum)):
    s_sum[i].add_marker(hs.plot.markers.vertical_line(direct[i]))

In [ ]:
ROI_signal = []
for s in signals:
    p_roi, p_roi_s = set_ROI(s,shape="circle")
    ROI_signal.append(p_roi_s)

In [ ]:
hs.plot.plot_spectra(ROI_signal, legend='auto', legend_loc='best')